### Import

In [1]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr


# import personal modules

# Path to modules
sys.path.append('../modules')

# Import my modules


In [2]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                                      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

### Landslide DF

In [3]:
def expand_grid(lat,lon):
    '''list all combinations of lats and lons using expand_grid(lat,lon)'''
    test = [(A,B) for A in lat for B in lon]
    test = np.array(test)
    test_lat = test[:,0]
    test_lon = test[:,1]
    full_grid = pd.DataFrame({'lat': test_lat, 'lon': test_lon})
    full_grid = full_grid.sort_values(by=['lat','lon'])
    full_grid = full_grid.reset_index(drop=True)
    return full_grid

In [4]:
fname = path_to_data + 'CH2_generated_data/Global_Landslide_Catalog_Export.csv'
landslide = pd.read_csv(fname)

# Select lat/lon grid
lonmin = 65
lonmax = 100
latmin = 20
latmax = 42

## Select Landslides within Southern Asia region
idx = (landslide.latitude >= latmin) & (landslide.latitude <= latmax) & (landslide.longitude >= lonmin) & (landslide.longitude <= lonmax)
landslide = landslide.loc[idx]
# set event time as index
landslide = landslide.set_index(pd.to_datetime(landslide.event_date))
# landslide.index = landslide.index.normalize()

# select only landslide dates that are between december and may
idx = (landslide.index.month >= 12) | (landslide.index.month <= 5)
landslide = landslide[idx]

# rename and reindex
landslide = landslide.rename(columns={"latitude": "lat", "longitude": "lon", "event_date": "event_time"})
landslide = landslide.reset_index()

# round event time to the nearest 6 hours
landslide['time'] = landslide['event_date'].dt.round('6H')
landslide = landslide.set_index(pd.to_datetime(landslide.time))

# select only landslide dates that are between december and may
idx = (landslide.index.month >= 12) | (landslide.index.month <= 5)
landslide = landslide[idx]

landslide

,event_date,source_name,source_link,event_id,event_time,event_time,event_title,event_description,location_description,location_accuracy,...,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,submitted_date,created_date,last_edited_date,lon,lat,time
time,,,,,,,,,,,,,,,,,,,,,
2007-03-19 00:00:00,2007-03-19 00:00:00,Tribune India.com,NaN,37,03/19/2007 12:00:00 AM,NaN,Doba,"2 killed, 5 injured",Doba,25km,...,Kashmir,14711.0,Kupwāra,18.10887,04/01/2014 12:00:00 AM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,74.333300,33.877800,2007-03-19 00:00:00
2017-05-28 12:00:00,2017-05-28 13:34:00,AGU Blogs (Petley),http://blogs.agu.org/landslideblog/2017/05/30/...,9774,05/28/2017 01:34:00 PM,NaN,Mudflow in Northern Tajikistan,"Meltwater causes large mudflow, affects cows",Rasht Valley of Tajikistan (exact coordinates ...,exact,...,NaN,NaN,NaN,NaN,06/16/2017 01:34:00 PM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,71.275479,39.197800,2017-05-28 12:00:00
2017-05-29 12:00:00,2017-05-29 13:34:00,Eleven Myanmar,http://www.elevenmyanmar.com/local/9759,9771,05/29/2017 01:34:00 PM,NaN,"Hakhha Landslide, Zayhuang Ward","Heavy rains cause landslides, affect houses mi...",Hakha in Zayhuang Ward,5km,...,NaN,NaN,NaN,NaN,06/16/2017 01:34:00 PM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,93.615899,22.656202,2017-05-29 12:00:00
2017-05-20 18:00:00,2017-05-20 20:14:00,World Hindu News,http://www.worldhindunews.com/2017/05/23/58870...,9802,05/20/2017 08:14:00 PM,NaN,Landslide in Chanoli district,"Boulders roll down mountainside, blocking highway","Rishikesh-Badrinath highway, near Vishnuprayag...",5km,...,NaN,NaN,NaN,NaN,06/20/2017 08:14:00 PM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,79.561632,30.565047,2017-05-20 18:00:00
2017-05-20 18:00:00,2017-05-20 20:14:00,Etemaa Daily,http://www.en.etemaaddaily.com/World/National/...,9809,05/20/2017 08:14:00 PM,NaN,Badrinath highway landslide,Landslide on highway to Hindu temple Badrinath...,"Highway to Badrinath, Uttarakhand",25km,...,NaN,NaN,NaN,NaN,06/20/2017 08:14:00 PM,11/20/2017 03:17:00 PM,02/15/2018 03:51:00 PM,79.492316,30.733156,2017-05-20 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-04-23 18:00:00,2017-04-23 17:58:00,Chandigarh Metro,https://chandigarhmetro.com/manali-rohtang-hig...,10853,04/23/2017 05:58:00 PM,NaN,Landslide near Rani Nullah,"Landslide blocks Malai-Rohtang HWY. Article: ""...","Manali Sarchu Road, Manali Sub-District, Kullu...",1km,...,NaN,NaN,NaN,NaN,07/17/2017 05:58:00 PM,12/05/2017 06:57:00 PM,02/15/2018 03:51:00 PM,77.233987,32.361492,2017-04-23 18:00:00
2017-04-17 06:00:00,2017-04-17 04:00:00,AKI Press,http://akipress.com/news:591338/,10897,04/17/2017 04:00:00 AM,NaN,Landslide in Chyrmash,Landslide hits 3 houses,"Chyrmash (??), Ozgon, Osh, Kyrgyzstan",50km,...,NaN,NaN,NaN,NaN,07/28/2017 01:34:00 PM,12/07/2017 09:19:00 PM,02/15/2018 03:51:00 PM,73.604732,40.804379,2017-04-17 06:00:00
2017-03-25 18:00:00,2017-03-25 17:32:00,Greater Kashmir,http://www.greaterkashmir.com/news/jammu/lands...,10845,03/25/2017 05:32:00 PM,NaN,Barnari Sigdi Landslide,Two teenage girls died after they were buried ...,"Barnari Sigdi area, Tehsil Mughalmaidan, Kisht...",5km,...,NaN,NaN,NaN,NaN,09/21/2017 05:32:00 PM,12/05/2017 06:45:00 PM,02/15/2018 03:51:00 PM,75.680611,33.403080,2017-03-25 18:00:00


In [5]:
# create grid for landslide data
full_grid = expand_grid(landslide.lat, landslide.lon)
full_grid

# merge landslide df with df of lats/lons
landslide = landslide.rename(columns={"time": "time_old"})
landslide = landslide.reset_index()
data_onto_full_grid = pd.merge(full_grid, landslide, how='left')
data_onto_full_grid 

# test to see that the rows with landslide data are there
idx = data_onto_full_grid.event_id > 0
test = data_onto_full_grid.loc[idx]

# make new dataframe with just lat, lon, event_date, and event_id
df = pd.concat([test.lat, test.lon, test.time, test.event_id],axis=1)
df



,lat,lon,time,event_id
424,20.848600,92.450400,2010-05-24 00:00:00,1899.0
846,20.944738,85.156627,2017-03-27 18:00:00,10954.0
1387,21.520200,91.969600,2012-03-08 06:00:00,4248.0
1922,22.130600,95.548800,2015-12-17 00:00:00,9073.0
2264,22.133700,78.897900,2016-03-07 00:00:00,9351.0
...,...,...,...,...
233843,41.254961,72.438099,2017-04-30 12:00:00,10908.0
234346,41.343100,72.929300,2016-05-15 00:00:00,9345.0
234824,41.408800,72.828200,2011-05-11 00:00:00,3483.0
235305,41.601407,72.677118,2017-05-17 12:00:00,9822.0


In [6]:
## pull unique lats and lons from list to make grid
out_lat = data_onto_full_grid.lat.unique()
out_lon =  data_onto_full_grid.lon.unique()
print(len(out_lat), len(out_lon))
print(len(out_lat)*len(out_lon))

# 6-hourly dates from 1979-12-01 to 2019-02-28
da_time = pd.date_range(start='1979-12-01', end='2019-05-31', freq='6H')
idx = (da_time.month >= 12) | (da_time.month <= 5)
new_time = da_time[idx]

## make an empty xarray dataset with the lat/lon grid from above
# put into a dataset
data_empty = np.empty([len(new_time), len(out_lat), len(out_lon)], dtype='uint16')
var_dict = {'landslide': (['time', 'lat', 'lon'], data_empty)}
ds = xr.Dataset(var_dict,
                coords={'time': (['time'], new_time),
                        'lat': (['lat'], out_lat),
                        'lon': (['lon'], out_lon)})

## iterate through the rows to assign the event_id to the empty xr dataset
## at the appropriate lat, lon, and time
## this is really only okay because we have a small amount of rows (e.g. < 1000)
for i, row in df.T.iteritems():
    ds["landslide"].loc[dict(lat = row['lat'], lon = row['lon'], time = row['time'])] = row['event_id']
    
# test to check first value
print(ds.sel(lat = 20.848600, lon = 92.450400, time = '2010-05-24 00:00:00'))


481 477
229437
<xarray.Dataset>
Dimensions:    ()
Coordinates:
    time       datetime64[ns] 2010-05-24
    lat        float64 20.85
    lon        float64 92.45
Data variables:
    landslide  uint16 1899


In [8]:
# now we want to see if there is an AR present at the same time and location as the landslides
# open the trackID for ARs
filename =  path_to_data + 'ar_catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ar = xr.open_dataset(filename, engine='netcdf4')
ar = ar.squeeze()

# Select months
idx = (ar.time.dt.month >= 12) | (ar.time.dt.month <= 5)
kid = ar.kidmap.sel(time=idx) # trackID for indexing

# slice the dates so both ds match
kid = kid.sel(time=slice('1979-12-01 00', '2019-05-31 00:00'))
kid

<xarray.DataArray 'kidmap' (time: 29157, lat: 121, lon: 240)>
[846719280 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 -90.0 -88.5 -87.0 -85.5 -84.0 ... 85.5 87.0 88.5 90.0
    lev      float64 1e+03
  * time     (time) datetime64[ns] 1979-12-01 1979-12-01T06:00:00 ... 2019-05-31
    ens      float64 1.0
Attributes:
    long_name:  Track ID
    units:      none

In [29]:
## for each landslide_id, if the lat/lon falls within an AR, keep that AR ID and landslide ID
landslideID = []
arID = []
for i, row in df.T.iteritems():
    t = kid.sel(lat = row['lat'], lon = row['lon'], time = row['time'], method='nearest').values
    print(t)
    if t != np.nan:
        landslideID.append(row['event_id'])
        arID.append(t)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
108408.0
nan
nan
111907.0
nan
nan
nan
nan
nan
111907.0
nan
nan
nan
nan
111907.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
98243.0
nan
nan
nan
nan
nan
nan
nan
98546.0
98379.0
98479.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
98379.0
98379.0
118455.0
nan
nan
95335.0
95335.0
95335.0
nan
nan
nan
nan
98633.0
98633.0
98633.0
98633.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
111907.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
104675.0
nan
nan
nan
100806.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
121814.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
97918.0
nan
101168.0
nan
na

In [10]:
## chunk the landslide ds so it doesn't take too much memory 
ds = ds.chunk({"time": 3000})
ds

<xarray.Dataset>
Dimensions:    (time: 29157, lat: 481, lon: 477)
Coordinates:
  * time       (time) datetime64[ns] 1979-12-01 ... 2019-05-31
  * lat        (lat) float64 20.85 20.94 21.52 22.13 ... 41.34 41.41 41.6 41.75
  * lon        (lon) float64 65.74 66.97 67.0 67.03 ... 97.8 97.85 99.29 99.45
Data variables:
    landslide  (time, lat, lon) uint16 dask.array<chunksize=(3000, 481, 477), meta=np.ndarray>

In [11]:
# regrid landslide ds to match the lats/lons of ar.shape 
## regrid/interpolate landslide data to have same grid size as ar data
regrid_lats = kid.lat
regrid_lons = kid.lon

ds = ds.interp(lon=regrid_lons, lat=regrid_lats)
ds


<xarray.Dataset>
Dimensions:    (time: 29157, lat: 121, lon: 240)
Coordinates:
  * time       (time) datetime64[ns] 1979-12-01 ... 2019-05-31
  * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
  * lat        (lat) float64 -90.0 -88.5 -87.0 -85.5 ... 85.5 87.0 88.5 90.0
    lev        float64 1e+03
    ens        float64 1.0
Data variables:
    landslide  (time, lat, lon) float64 dask.array<chunksize=(3000, 121, 240), meta=np.ndarray>

In [12]:
# merge together ar.shape and landslide_ds
ds = ds.assign(ar=lambda ds: kid)
ds

<xarray.Dataset>
Dimensions:    (time: 29157, lat: 121, lon: 240)
Coordinates:
  * time       (time) datetime64[ns] 1979-12-01 ... 2019-05-31
  * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
  * lat        (lat) float64 -90.0 -88.5 -87.0 -85.5 ... 85.5 87.0 88.5 90.0
    lev        float64 1e+03
    ens        float64 1.0
Data variables:
    landslide  (time, lat, lon) float64 dask.array<chunksize=(3000, 121, 240), meta=np.ndarray>
    ar         (time, lat, lon) float64 ...

In [13]:
ds = ds.chunk({"time": 3000})
ds

<xarray.Dataset>
Dimensions:    (time: 29157, lat: 121, lon: 240)
Coordinates:
  * time       (time) datetime64[ns] 1979-12-01 ... 2019-05-31
  * lon        (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 354.0 355.5 357.0 358.5
  * lat        (lat) float64 -90.0 -88.5 -87.0 -85.5 ... 85.5 87.0 88.5 90.0
    lev        float64 1e+03
    ens        float64 1.0
Data variables:
    landslide  (time, lat, lon) float64 dask.array<chunksize=(3000, 121, 240), meta=np.ndarray>
    ar         (time, lat, lon) float64 dask.array<chunksize=(3000, 121, 240), meta=np.ndarray>

In [16]:
ds.lon.values

array([  0. ,   1.5,   3. ,   4.5,   6. ,   7.5,   9. ,  10.5,  12. ,
        13.5,  15. ,  16.5,  18. ,  19.5,  21. ,  22.5,  24. ,  25.5,
        27. ,  28.5,  30. ,  31.5,  33. ,  34.5,  36. ,  37.5,  39. ,
        40.5,  42. ,  43.5,  45. ,  46.5,  48. ,  49.5,  51. ,  52.5,
        54. ,  55.5,  57. ,  58.5,  60. ,  61.5,  63. ,  64.5,  66. ,
        67.5,  69. ,  70.5,  72. ,  73.5,  75. ,  76.5,  78. ,  79.5,
        81. ,  82.5,  84. ,  85.5,  87. ,  88.5,  90. ,  91.5,  93. ,
        94.5,  96. ,  97.5,  99. , 100.5, 102. , 103.5, 105. , 106.5,
       108. , 109.5, 111. , 112.5, 114. , 115.5, 117. , 118.5, 120. ,
       121.5, 123. , 124.5, 126. , 127.5, 129. , 130.5, 132. , 133.5,
       135. , 136.5, 138. , 139.5, 141. , 142.5, 144. , 145.5, 147. ,
       148.5, 150. , 151.5, 153. , 154.5, 156. , 157.5, 159. , 160.5,
       162. , 163.5, 165. , 166.5, 168. , 169.5, 171. , 172.5, 174. ,
       175.5, 177. , 178.5, 180. , 181.5, 183. , 184.5, 186. , 187.5,
       189. , 190.5,

In [21]:
# test to check first value
print(ds.landslide.sel(time = '2010-05-24 00:00:00').max().values)

0.0


In [26]:
print(ds.ar.sel(lat = 20.848600, lon = 92.450400, time = '2010-05-24 00:00:00', method='nearest').values)

nan


In [ ]:
# for each landslide_id:
# if landslide > 0 & ar > 0, save landslide_id 

### AR Type Data

In [3]:
# filepath = path_to_out + 'DJFMAM_ARS_TRACKID_daily.csv'
filepath = path_to_out + 'HMAAR_duration_DJFMAM.csv' # just dates in HMA
# filepath = path_to_out + 'HMA_AR_duration.csv' # dates based on AR Catalog
df = pd.read_csv(filepath)
df = df.drop(columns=['Unnamed: 0'])
df

,trackID,ar_cat,start_date,end_date,duration
0,2861.0,1,1979-12-02 00:00:00,1979-12-02 18:00:00,18.0
1,2871.0,1,1979-12-09 06:00:00,1979-12-09 18:00:00,12.0
2,2975.0,1,1979-12-16 12:00:00,1979-12-17 00:00:00,12.0
3,2988.0,1,1979-12-21 00:00:00,1979-12-22 12:00:00,36.0
4,3026.0,1,1979-12-24 06:00:00,1979-12-24 12:00:00,6.0
...,...,...,...,...,...
2684,128613.0,3,2019-05-06 06:00:00,2019-05-06 12:00:00,6.0
2685,128655.0,3,2019-05-12 06:00:00,2019-05-12 12:00:00,6.0
2686,128755.0,3,2019-05-24 00:00:00,2019-05-24 06:00:00,6.0
2687,128763.0,3,2019-05-25 00:00:00,2019-05-25 06:00:00,6.0


### AR Data

In [6]:
## duration of ARs based on trackID
# for each AR Track ID, find the duration of that track
filename =  path_to_data + 'ar_catalog/globalARcatalog_ERA-Interim_1979-2019_v3.0.nc'
ds = xr.open_dataset(filename, engine='netcdf4')
ds = ds.squeeze()

# Select months
idx = (ds.time.dt.month >= 12) | (ds.time.dt.month <= 5)
kivty = ds.kivty.sel(time=idx) # ivty for each trackID
kivtx = ds.kivtx.sel(time=idx) # ivtx for each trackID
kid = ds.kid.sel(time=idx) # trackID for indexing
hlat = ds.hlat.sel(time=idx) # head latitude of each trackID
hlon = ds.hlon.sel(time=idx) # head longitude of each trackID
tlat = ds.tlat.sel(time=idx) # tail latitude of each trackID
tlon = ds.tlon.sel(time=idx) # tail longitude of each trackID
width = ds.width.sel(time=idx) # width of AR

In [7]:
%%time
# append ivty and ivtx to df for each trackID
ivty = []
ivtx = []
trID = []
trLAT = []
trLON = []
tlLAT = []
tlLON = []
ar_width = []

for i, track in enumerate(df.trackID.values):
    idx = kivtx.where(kid == track, drop=True).values
    x = idx[~np.isnan(idx)][0]
    idy = kivty.where(kid == track, drop=True).values
    y = idy[~np.isnan(idy)][0]
    
    # append to lists
    ivtx.append(x)
    ivty.append(y)
    trID.append(track)
    
    # head lat and lon of each AR
    idlat = hlat.where(kid == track, drop=True).values
    trLAT.append(np.asarray(idlat[~np.isnan(idlat)]).mean())
    idlon = hlon.where(kid == track, drop=True).values
    trLON.append(np.asarray(idlon[~np.isnan(idlon)]).mean())
    
    # tail lat and lon of each AR
    idlat = tlat.where(kid == track, drop=True).values
    tlLAT.append(np.asarray(idlat[~np.isnan(idlat)]).mean())
    idlon = tlon.where(kid == track, drop=True).values
    tlLON.append(np.asarray(idlon[~np.isnan(idlon)]).mean())
    
    # avg width of each AR
    idwidth = width.where(kid == track, drop=True).values
    ar_width.append(np.asarray(idwidth[~np.isnan(idwidth)]).mean())

CPU times: user 12min 23s, sys: 1.62 s, total: 12min 25s
Wall time: 12min 26s


In [8]:
# put collected data in df
df['ivty'] = ivty
df['ivtx'] = ivtx
df['ivt'] = np.sqrt(df.ivty**2 + df.ivtx**2)
df['hlat'] = trLAT
df['hlon'] = trLON
df['tlat'] = tlLAT
df['tlon'] = tlLON
df['width'] = ar_width
df

,trackID,ar_cat,start_date,end_date,duration,ivty,ivtx,ivt,hlat,hlon,tlat,tlon,width
0,2861.0,1,1979-12-02 00:00:00,1979-12-02 18:00:00,18.0,84.715817,156.934744,178.340359,54.000000,88.950000,33.300000,54.750000,809390.401868
1,2871.0,1,1979-12-09 06:00:00,1979-12-09 18:00:00,12.0,140.456836,255.515681,291.575695,58.855263,137.842105,37.657895,165.789474,818381.988156
2,2975.0,1,1979-12-16 12:00:00,1979-12-17 00:00:00,12.0,123.568145,129.182257,178.765606,40.000000,64.000000,24.000000,58.500000,541836.077579
3,2988.0,1,1979-12-21 00:00:00,1979-12-22 12:00:00,36.0,118.158087,138.478370,182.037338,33.600000,56.800000,16.600000,45.300000,648660.972048
4,3026.0,1,1979-12-24 06:00:00,1979-12-24 12:00:00,6.0,95.302288,105.452260,142.136221,28.500000,69.000000,19.500000,49.500000,501187.836362
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,128613.0,3,2019-05-06 06:00:00,2019-05-06 12:00:00,6.0,152.255674,36.972662,156.680465,40.500000,102.000000,28.500000,94.500000,545764.686818
2685,128655.0,3,2019-05-12 06:00:00,2019-05-12 12:00:00,6.0,92.532689,210.106101,229.579773,30.000000,100.500000,28.500000,84.000000,304775.576674
2686,128755.0,3,2019-05-24 00:00:00,2019-05-24 06:00:00,6.0,59.469964,404.299149,408.649579,24.000000,100.500000,21.000000,82.500000,233469.625341
2687,128763.0,3,2019-05-25 00:00:00,2019-05-25 06:00:00,6.0,269.018570,280.446504,388.614504,44.307692,140.653846,24.115385,113.653846,545386.913129


### Subregions to calculate precipitation

In [9]:
# latmin, latmax, lonmin, lonmax
ext1 = [69, 74, 37, 40] # Northwestern precip anomalies
ext2 = [71, 79, 32, 37] # Western precip anomalies
ext3 = [90, 99, 24, 30] # Eastern precip anomalies
ext4 = [48, 53, 30, 35] # zagros mountains precip anomalies
region_name = ['northwestern', 'western', 'eastern', 'zagros']
domains = [ext1, ext2, ext3, ext4]

# domain extent info 
# [xmin, ymin]
sr_xy = []
sr_width = []
sr_height = []

for i, d in enumerate(domains):
    sr_xy.append([d[0], d[2]])
    sr_width.append((d[1] - d[0]))
    sr_height.append((d[3]- d[2]))

print(sr_xy)
print(sr_width)
print(sr_height)


[[69, 37], [71, 32], [90, 24], [48, 30]]
[5, 8, 9, 5]
[3, 5, 6, 5]


### Link Landslide to AR DF

In [8]:
## link up landslide AR df with above
fname = path_to_out + 'djfmam_landslide_ars.csv'
landslide = pd.read_csv(fname)

ar_trackID = df.trackID
landslide_tracks = landslide.kidmap.values
landslide_lats = landslide.latitude.values
landslide_lons = landslide.longitude.values
arr_allDays = np.zeros(len(ar_trackID), dtype=int)
arr_latitude = np.zeros(len(ar_trackID), dtype=float)
arr_longitude = np.zeros(len(ar_trackID), dtype=float)

# Loop over ar days and match to ar_full 
for i, (track, lat, lon) in enumerate(zip(landslide_tracks, landslide_lats, landslide_lons)):
    idx = np.where(ar_trackID == track)
    arr_allDays[idx] = 1
    # add in landslide lat and lon
    arr_latitude[idx] = lat
    arr_longitude[idx] = lon
df['landslide'] = arr_allDays
df['landslide_lat'] = arr_latitude
df['landslide_lon'] = arr_longitude
df


,trackID,ar_cat,start_date,end_date,duration,ivty,ivtx,ivt,hlat,hlon,tlat,tlon,landslide,landslide_lat,landslide_lon
0,2861.0,1,1979-12-02 00:00:00,1979-12-02 18:00:00,18.0,84.715817,156.934744,178.340359,54.000000,88.950000,33.300000,54.750000,0,0.0,0.0
1,2871.0,1,1979-12-09 06:00:00,1979-12-09 18:00:00,12.0,140.456836,255.515681,291.575695,58.855263,137.842105,37.657895,165.789474,0,0.0,0.0
2,2975.0,1,1979-12-16 12:00:00,1979-12-17 00:00:00,12.0,123.568145,129.182257,178.765606,40.000000,64.000000,24.000000,58.500000,0,0.0,0.0
3,2988.0,1,1979-12-21 00:00:00,1979-12-22 12:00:00,36.0,118.158087,138.478370,182.037338,33.600000,56.800000,16.600000,45.300000,0,0.0,0.0
4,3026.0,1,1979-12-24 06:00:00,1979-12-24 12:00:00,6.0,95.302288,105.452260,142.136221,28.500000,69.000000,19.500000,49.500000,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,128613.0,3,2019-05-06 06:00:00,2019-05-06 12:00:00,6.0,152.255674,36.972662,156.680465,40.500000,102.000000,28.500000,94.500000,0,0.0,0.0
2685,128655.0,3,2019-05-12 06:00:00,2019-05-12 12:00:00,6.0,92.532689,210.106101,229.579773,30.000000,100.500000,28.500000,84.000000,0,0.0,0.0
2686,128755.0,3,2019-05-24 00:00:00,2019-05-24 06:00:00,6.0,59.469964,404.299149,408.649579,24.000000,100.500000,21.000000,82.500000,0,0.0,0.0
2687,128763.0,3,2019-05-25 00:00:00,2019-05-25 06:00:00,6.0,269.018570,280.446504,388.614504,44.307692,140.653846,24.115385,113.653846,0,0.0,0.0


### Export to CSV

In [12]:
# Export dataframe as csv
outfile = path_to_out + 'DJFMAM_ivt_ar_types.csv'     
df.to_csv(outfile)